
<center><img src="https://raw.githubusercontent.com/dssg/aequitas/master/src/aequitas_webapp/static/images/aequitas_flow_header.svg"></center>

# Correcting the predictions of a Model

In this notebook we will first **load a Machine Learning model** created through an **`Experiment`** of **Aequitas Flow**. We will  measure its performance and run a **fairness audit**  using the application-specific configurations.

We will then apply a **post-processing method to correct the predictions**, and observe any **changes in fairness** and **performance**.

---
## Initial Setup

This section covers the initial setup required for the notebook. We'll be **installing the most recent version of Aequitas**.

> ⚠️ **This notebook assumes that an ML Model has already been trained**. ⚠️

We'll also be retrieving the model pickle file from a previous experiment, downloading it directly from the [Aequitas Repository](https://github.com/dssg/aequitas/tree/master/examples). However, the notebook supports the use of other models or datasets.

In [1]:
# Install Aequitas (change to version install after release)
!pip install git+https://github.com/dssg/aequitas.git@release-fixes &> /dev/null
# This only needs to run once, or after your runtime environment gets deleted.

In [2]:
# This will avoid double logging in Colab
from aequitas.flow.utils.logging import clean_handlers

clean_handlers()

In [3]:
# This cell will download a model from the repository. You do not need to run it if you have your won model.
from aequitas.flow.utils.colab import get_examples

get_examples("experiment_results")

[INFO] 2024-01-12 09:16:18 utils.colab - Downloading examples from fairflow repository.
[INFO] 2024-01-12 09:16:39 utils.colab - Examples downloaded.


---
## Loading the model & datasets

In this section we will load the model for the audit and the evaluation datasets.

If you are testing your own model, make sure to send it to the Colab environment (or any other environment you are using this notebook on).

Starting with the model:

In [4]:
pickle_path = "examples/experiment_results/lgbm_baf_sample.pickle"

In [5]:
# Change this cell if your model is loaded in a different form.
import pickle

with open(pickle_path, "rb") as f:
    model = pickle.load(f)

In this example, we are using a sample of the **BankAccountFraud** dataset. This dataset presents a predictive task of detecting fraudulent attempts of bank account opening.

In case you want to use a different dataset, make sure it is loaded as a pandas dataframe. If possible, configure an `aequitas.flow.datasets.GenericDataset`, for less changes in other cells.

Now we will load the dataset:

In [6]:
from aequitas.flow.methods.base_estimator import LightGBM
from aequitas.flow.datasets import BankAccountFraud

dataset = BankAccountFraud("Sample")
dataset.load_data()
dataset.create_splits()

validation = dataset.validation
test = dataset.test

[INFO] 2024-01-12 09:16:46 datasets.BankAccountFraud - Instantiating a BankAccountFraud dataset.
[INFO] 2024-01-12 09:16:46 datasets.BankAccountFraud - Downloading sample data from repository.
[INFO] 2024-01-12 09:16:47 datasets.BankAccountFraud - Downloaded data successfully.
[INFO] 2024-01-12 09:16:47 datasets.BankAccountFraud - Loading data from /usr/local/lib/python3.10/dist-packages/aequitas/datasets/BankAccountFraud/Sample.parquet


---
## Obtaining the predictions and thresholding the model

In the following cells, we will generate the predictions with the model and the dataset, and create a threshold for it.

In [7]:
# If your model is not from
preds_val = model.predict_proba(validation.X, validation.s)
preds_test = model.predict_proba(test.X, test.s)

We use the `Threshold` from Aequitas Flow for thresholding. In the specific use-case of the BankAccountFraud dataset, we are pointing for an FPR value of 5%. You can adjust that by changing the instantiation of this class.

The Threshold will be fitted to a validation set, and used to binarize the test set. Note that you can threshold the same test you fit, but this might lead to overfitting.

In [8]:
from aequitas.flow.methods.postprocessing import Threshold

# We will create a threshold based to obtain 5% FPR on validation
threshold = Threshold(threshold_type="fpr", threshold_value=0.05)
threshold.fit(validation.X, preds_val, validation.y, validation.s)

print(f"Threshold Value: {threshold.threshold}")

[INFO] 2024-01-12 09:16:48 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.
[INFO] 2024-01-12 09:16:48 methods.postprocessing.Threshold - Computing threshold.
[INFO] 2024-01-12 09:16:48 methods.postprocessing.Threshold - Finished computing threshold.
Threshold Value: 0.031104218027432882


In [9]:
# Binarize test predictions with previously calculated threshold
bin_preds_test = threshold.transform(test.X, preds_test, test.s)

[INFO] 2024-01-12 09:16:48 methods.postprocessing.Threshold - Transforming predictions.
[INFO] 2024-01-12 09:16:48 methods.postprocessing.Threshold - Finished transforming predictions.


---
## Fairness Audit and Performance Evaluation

Now, we will create the resources necessary to perform a fairness audit, and evaluate the model performance. These are:
1. **Protected attribute**
2. **Model predictions**
3. **Labels**

But first, we will have to define some configurations of this step.

As a brief summary of the task of the BankAccountFraud dataset, the performance is determined by the percentage of positive instances (frauds) detected (TPR). False Positives will incur in a non-fraudulent individual not having a bank account due to a false flag. Because of this, we want to equalize the rate of false positives (FPR) in relation to the protected attribute, in this case the customer age. The reference group for this task is the group with younger age (<50).

> ⚠️ **Make sure to update the following configuration cell with the appropriate values for your use-case**. ⚠️

In [10]:
performance_metric = "tpr"
fairness_metric = "fpr"

# The column name of the sensitive attribute can be obtained from the aequitas.flow.Dataset object
fairness_column = test.s.name
# The reference group for the example of BAF is "0", i.e. individuals from the younger group (<50).
reference_group = "0"

In [11]:
# Creating a dataframe for the fairness audit
audit_df = test.s.astype(str).to_frame()
audit_df["label"] = test.y
# These might need to change if you are not using an aequitas.flow.Dataset object

audit_df["score"] = bin_preds_test

In the cell bellow we will see the minimal structure for a fairness audit DataFrame:

In [12]:
audit_df.sample(n=5, random_state=2)

,customer_age_bin,label,score
92592,0,0,0
88551,1,0,0
94698,0,0,0
88759,0,0,1
87353,0,0,0


We will quickly observe the performance of the model with a method for the effect.

In this dataset, we are using global **TPR** as performance metric.

In [13]:
from aequitas.flow.evaluation import evaluate_performance
import pandas as pd

pd.Series(evaluate_performance(audit_df["label"], audit_df["score"]))[performance_metric]

0.5150214592274678

We will now perform the fairness audit:

In [14]:
from aequitas.group import Group
from aequitas.plot import summary

g = Group()
disparities = g.calculate_disparities(audit_df, "score", "label", [fairness_column], {fairness_column: reference_group})

summary(disparities, [fairness_metric])

alt.VConcatChart(...)

In [15]:
from aequitas.plot import disparity

disparity(disparities, [fairness_metric], attribute=fairness_column)

alt.LayerChart(...)

In [24]:
disparities[["attribute_name", "attribute_value", "tpr", "fpr", "accuracy", "precision"]]

,attribute_name,attribute_value,tpr,fpr,accuracy,precision
0,customer_age_bin,0,0.467456,0.040324,0.953272,0.132550
1,customer_age_bin,1,0.640625,0.127955,0.864677,0.141379


---
## Correcting the predictions

To correct the predictions, we will use a method available in Aequitas Flow.

This method will calculate different thresholds to equalize a target metric for all the groups (in this case the fairness metric, FPR).

In [16]:
from aequitas.flow.methods.postprocessing import GroupThreshold

threshold = GroupThreshold("fpr", 0.05)

threshold.fit(validation.X, preds_val, validation.y, validation.s)

[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.GroupThreshold - Instantiating postprocessing GroupThreshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Computing threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Finished computing threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Computing threshold.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Finished computing threshold.


In [17]:
corrected_bin_preds_test = threshold.transform(test.X, preds_test, test.s)

[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Transforming predictions.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Finished transforming predictions.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Transforming predictions.
[INFO] 2024-01-12 09:16:50 methods.postprocessing.Threshold - Finished transforming predictions.


In [18]:
audit_df = test.s.astype(str).to_frame().copy()
audit_df["score"] = corrected_bin_preds_test
audit_df["label"] = test.y

Let's see the impact of this correction in the global recall of the model, with the updated binarized predictions:

In [19]:
pd.Series(evaluate_performance(audit_df["label"], audit_df["score"]))[performance_metric]

0.5150214592274678

When compared to the previous value of recall, there is no drop in performance.

Let's observe the fairness audit:

In [20]:
g = Group()
disparities_fixed = g.calculate_disparities(audit_df, "score", "label", [fairness_column], {fairness_column: reference_group})

summary(disparities_fixed, [fairness_metric])

alt.VConcatChart(...)

In [21]:
from aequitas.plot import disparity

disparity(disparities_fixed, [fairness_metric], attribute=fairness_column)

alt.LayerChart(...)

In [25]:
disparities_fixed[["attribute_name", "attribute_value", "tpr", "fpr", "accuracy", "precision"]]

,attribute_name,attribute_value,tpr,fpr,accuracy,precision
0,customer_age_bin,0,0.538462,0.060292,0.934488,0.105324
1,customer_age_bin,1,0.453125,0.062693,0.921891,0.192053
